# **Using Ollama**

In [40]:
import requests
import json
import sys
import time

def query_ollama_stream(keywords):
    """Interact with the Ollama server using streaming HTTP responses and overwrite output."""
    url = "http://localhost:11434/api/chat"  # Correct endpoint
    prompt = f"""
    Here is a list of keywords extracted from a job description:
    {', '.join(keywords)}

    Please return an array of keywords that represent professional skills or 
    technologies suitable for inclusion in a resume without any explanation and additonal text.
    """
    payload = {
        "model": "llama3.1",  # Replace with your desired model name
        "messages": [  # Correct key
            {
                "role": "user",
                "content": prompt
            }
        ]
    }
    headers = {
        "Content-Type": "application/json"
    }

    try:
        # Set stream=True to handle streamed responses
        with requests.post(url, json=payload, headers=headers, stream=True) as response:
            response.raise_for_status()  # Raise an HTTPError for bad responses

            full_response = ""  # Initialize a variable to store the complete response
            for line in response.iter_lines():
                if line:  # Skip empty lines
                    try:
                        # Decode the JSON line
                        json_line = json.loads(line.decode("utf-8"))
                        # Extract the assistant's content
                        content = json_line.get("message", {}).get("content", "")
                        if content:
                            full_response += content  # Append new content
                            # # Overwrite the line with the updated response
                            # sys.stdout.write(f"\r{full_response}")
                            # sys.stdout.flush()
                    except json.JSONDecodeError as e:
                        print(f"Error decoding JSON line: {e}")

            print()  # Move to a new line after the response is complete
            return full_response
    except requests.exceptions.RequestException as e:
        raise Exception(f"Error querying Ollama: {e}")


# **Using Openapi APIs**

In [65]:

import os
from openai import OpenAI
import ast 


def query_openai_for_keywords(keywords):
    """
    Query OpenAI's API to identify professional keywords using the updated method.
    """
    # Load API key from environment variable
    openai_api_key = os.getenv("OPENAI_API_KEY")
    if not openai_api_key:
        raise Exception("OPENAI_API_KEY is not set in the environment variables.")
    
    # Initialize OpenAI client with the API key
    client = openai.Client(api_key=openai_api_key)

    # Construct the prompt
    prompt = f"""
    Filter the following list for professional skills, technologies, or competencies:
    {', '.join(keywords)}.
    Return the result as a plain Python list of strings.
    """

    try:
        # Call OpenAI API using client.chat.completions.create
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ]
        )
        # Extract the response content
        # print("Raw Response Content:", response)
        content = response.choices[0].message.content.strip()
        # Safely evaluate the returned Python list
        professional_keywords = ast.literal_eval(content)
        return professional_keywords
    except Exception as e:
        raise Exception(f"Error querying OpenAI: {e}")

# **Extracting keywords with NLP**

In [59]:
import spacy

# Load the spaCy language model
nlp = spacy.load('en_core_web_sm')

def process_job_description(file_path):
    # Step 1: Read the contents of the offer.txt file
    with open(file_path, 'r') as file:
        job_description = file.read()
    
    doc = nlp(job_description)
    keywords = [token.text for token in doc if token.is_alpha and not token.is_stop]
    return set(keywords)
    



# **Main**

In [66]:
# Example usage
if __name__ == "__main__":
    job_description_file = "offer.txt"  # Path to your job description file
    keywords = process_job_description(job_description_file)
    print(f"Extracted Keywords: {keywords}")

    professional_keywords = query_openai_for_keywords(keywords)
    print("\nProfessional Keywords for Resume:")
    print(professional_keywords)

Extracted Keywords: {'Collaboration', 'REST', 'aptitude', 'ensure', 'innovative', 'office', 'Java', 'Drafting', 'suppliers', 'Computer', 'support', 'SQL', 'Azure', 'solving', 'Developing', 'knowledge', 'Knowledge', 'fields', 'analysts', 'training', 'Engineering', 'Good', 'Developer', 'department', 'code', 'team', 'consultancy', 'message', 'science', 'Axpo', 'organised', 'relevant', 'Excellent', 'Ecosystem', 'new', 'Mathematics', 'Visualforce', 'learning', 'experience', 'carry', 'activities', 'complete', 'Italian', 'success', 'multinational', 'Research', 'release', 'effective', 'job', 'models', 'SOAP', 'orientation', 'optimization', 'area', 'cycle', 'APEX', 'microservices', 'appreciated', 'MuleSoft', 'colleagues', 'technologies', 'Microsoft', 'technical', 'analytical', 'design', 'Implementation', 'opportunity', 'Analysis', 'Communication', 'Italia', 'graduates', 'recent', 'computer', 'Delivery', 'functional', 'business', 'Solution', 'SW', 'process', 'modernly', 'documentation', 'skills'